# Segmenting and Clustering Neighborhoods in Toronto
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

In [7]:
# import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

print("Libraries imported")

Libraries imported


**Scrape the wikipedia page for Toronto neighborhood data and wrangle into a dataframe**

In [15]:
# grab the data from wikipediea
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(wiki_url).text
canada_xml = bs(source, 'lxml')
table=canada_xml.find('table')


In [16]:
# grab the columns Postalcode,Borough and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df_canada = pd.DataFrame(columns = column_names)


In [20]:
# build the table
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df_canada.loc[len(df_canada)] = row_data
        
df_canada.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Let's cleanup some data**

In [26]:
# Get rid of the not assigned boroughs
df_canada=df_canada[df_canada['Borough']!='Not assigned']
# Are there any neighborhoods not assigned?
(df_canada['Neighborhood'] == "Not assigned").any()


False

**We don't have any "Not Assigned" neighborhoods since the answer to the last answer is False.**

In [27]:
df_canada.shape

(206, 3)

**We have 206 rows and three columns.**